The next step is to test basic models with our data, as opposed to more complex models such as neural networks. I imported the necessary libraries below and then combined the different csv files that comprised of our total collected article data.

In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
import scipy
import zipfile

In [65]:
df = pd.read_json("../../data/News_Category_Dataset_v2.json", lines = True)


zf = zipfile.ZipFile('../../data/dffulltext.csv.zip') 
df_i = pd.read_csv(zf.open('dffulltext.csv'))
df_ii = pd.read_csv("../../data/1st_50k.gzip", compression = "gzip")
df_iii = pd.read_csv("../../data/2nd_50k.gzip", compression = "gzip")

In [66]:
df_i.head()

,Unnamed: 0,category,headline,authors,link,short_description,date,full_text
0,0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,Mei-Chun Jau for HuffPost Amanda Painter is th...
1,1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,The 2018 FIFA World Cup starts June 14 in Russ...
2,2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,Love actually turned to matrimony for Hugh Gra...
3,3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,Rep. Adam Schiff (D-Calif.) and fellow Democra...
4,4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,The “Dietland” star told host Jimmy Fallon tha...


In [67]:
df_i.drop(["Unnamed: 0", "authors", "link", "date", "short_description"], axis = 1, inplace = True)

In [68]:
df["category"].iloc[0]

'CRIME'

In [69]:
df_target_ii = []
df_target_iii = []
x = 100000
for i in df["category"].iloc[100001:]:
    if x < 150001:
        df_target_ii.append(df["category"].iloc[x])
    else:
        df_target_iii.append(df["category"].iloc[x])
    x += 1

In [70]:
df_ii["target"] = df_target_ii
df_iii["target"] = df_target_iii

In [71]:
df_iii.drop(['headline', "full_text"], axis = 1, inplace = True)
df_ii.drop(["headline", "full_text"], axis = 1, inplace = True)

In [72]:
df = pd.concat([df_ii, df_iii], axis = 0)

In [73]:
df_i["combined"] = df_i["headline"] + df_i["full_text"]
df_i["target"] = df_i["category"]
df_i.drop(["headline", "full_text", "category"], axis = 1, inplace = True)

In [74]:
df_final = pd.concat([df, df_i], axis = 0)

In [75]:
df_final.dropna(subset =["combined"], inplace = True)

In [93]:
df_final.reset_index(inplace = True)

In [94]:
df_final

,index,combined,target,cleaned
0,1,Immigration Advocates Try To Assuage Fears Ove...,POLITICS,
1,2,UVA Students Say Banning Sorority Members From...,POLITICS,
2,3,'Zombie Cat' Should Not Go Back To Owner Who B...,EDUCATION,
3,5,Autism Without Fear: Examining the Legitimacy ...,HEALTHY LIVING,
4,6,Being a Philanthropist Without MoneyPeople say...,HEALTHY LIVING,
...,...,...,...,...
129140,40378,Book Publishers Are Scrambling To Release Trum...,ARTS & CULTURE,
129141,40379,Second NFL Player Targeted By Stomach-Turning ...,DIVERSE VOICES,
129142,40380,Senate Democrats Give Up On Coal Miner Health ...,POLITICS,
129143,40381,Courtesy Over Death Penalty Cases May Be Dead ...,POLITICS,


In [77]:
df_final = df_final.loc[df_final["target"] != "WEIRD NEWS"]
f_final = df_final.loc[df_final["target"] != "IMPACT"]
df_final = df_final.loc[df_final["target"] != "GOOD NEWS"]
df_final = df_final.loc[df_final["target"] != "FIFTY"]

In [80]:
df_final.target = df_final.target.replace(["ARTS",'CULTURE & ARTS'], "ARTS & CULTURE").replace(['THE WORLDPOST'], 'WORLD NEWS').replace(['STYLE'], 'STYLE & BEAUTY')\
.replace(['TASTE'], 'FOOD & DRINK').replace(['WELLNESS'], 'HEALTHY LIVING').replace(['PARENTING'], 'PARENTS').replace(['MONEY'], 'BUSINESS')\
.replace(['COLLEGE'], 'EDUCATION').replace(['GREEN'], 'ENVIRONMENT').replace(["WOMEN", "BLACK VOICES", 'QUEER VOICES', 'LATINO VOICES'], 'DIVERSE VOICES')



In [96]:
pip install wn

     |████████████████████████████████| 66 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 62 kB 1.7 MB/s  eta 0:00:01
  Attempting uninstall: requests
    Found existing installation: requests 2.24.0
    Uninstalling requests-2.24.0:
      Successfully uninstalled requests-2.24.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

jupyter-server 1.9.0 requires tornado>=6.1.0, but you'll have tornado 6.0.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [99]:
from nltk.stem import WordNetLemmatizer

In [105]:
from tqdm import tqdm
import re
import wn
import nltk
nltk.download('wordnet')
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer

ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

pictwitter = df_final.combined.loc[df_final.combined.str.startswith("pictwitter")]
http = df_final.combined.loc[df_final.combined.str.contains("http")]
www = df_final.combined.loc[df_final.combined.str.startswith("www") == True]
stop = stopwords.words('english') + list(pictwitter) + list(http) + list(www)

def clean_history(history):
    history = re.sub('<a\b[^>]*>(.*?)</a>',
       '',history)
    punct_translator=str.maketrans('','',string.punctuation.replace('.', '') + '―“”’')
    digit_translator=str.maketrans('','',string.digits)
    history=history.translate(punct_translator)
    history=history.translate(digit_translator)
    split = history.split()
    history = " ".join([wn.lemmatize(word.lower()) for word in split if word.lower() not in stop and '.com' not in word])
#     tokens = re.split('\W+', history)
    #history = [snow_stemmer.stem(word) for word in tokens if word not in stop]
#     history = [wn.lemmatize(word) for word in tokens if word not in stop]
#     history = ' '.join(history)
    return history
df_final['cleaned'] = ''
with tqdm(total=len(df_final)) as pbar:
    for idx in df_final.index:
        df_final.at[idx, 'cleaned'] = clean_history(df_final.combined[idx])
        pbar.update(1)
#         df['text_clean'] = df['full_text'].apply(lambda x: clean_history(x))


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jerryhudspeth/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
100%|██████████| 129145/129145 [4:08:11<00:00,  8.67it/s]  


In [ ]:
sm = SMOTE(random_state=42)
#fit resample just training data. The testing observations presumably do not require smoting
X_tr_res, y_tr_res = sm.fit_resample(idf_train, y_tr)